In [1]:
import gym

In [2]:
env = gym.make('SpaceInvaders-v0')

In [3]:
episodes = 10

for episode in range(1, episodes):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        state, reward, done, info = env.step(env.action_space.sample())
        score += reward
    print('Episode: {}\nScore: {}'.format(episode, score))

Episode: 1
Score: 370.0
Episode: 2
Score: 195.0
Episode: 3
Score: 35.0
Episode: 4
Score: 185.0
Episode: 5
Score: 270.0
Episode: 6
Score: 415.0
Episode: 7
Score: 105.0
Episode: 8
Score: 320.0
Episode: 9
Score: 125.0


In [4]:
#import Neural Network Packages
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam


In [5]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation= 'relu'))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation= 'relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [6]:
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [13]:
del model

In [14]:
model = build_model(height, width, channels, actions)

In [15]:
#importing keras-rl2 reinforcement learning function
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy


In [16]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1, value_min=1, value_test=2, nb_steps=10000)
    memory = SequentialMemory(limit = 2000, window_length=3)
    dqn = DQNAgent(model=model,memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions = actions, nb_steps_warmup=1000)
    return dqn

In [17]:
dqn = build_agent(model, actions)

In [18]:
dqn.compile(Adam(lr=0.0001))

In [19]:
dqn.fit(env, nb_steps=40000, visualize=False, verbose=1)

Training for 40000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
10000/10000 [==============================] - 11507s 1s/step - reward: 0.3010
13 episodes - episode_reward: 227.308 [50.000, 470.000] - loss: 2.577 - mean_q: 7.569 - mean_eps: 1.000 - ale.lives: 1.921

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 43709s 4s/step - reward: 0.2575
13 episodes - episode_reward: 202.308 [65.000, 420.000] - loss: 1.978 - mean_q: 8.999 - mean_eps: 1.000 - ale.lives: 2.101

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 13218s 1s/step - reward: 0.2340
16 episodes - episode_reward: 146.250 [45.000, 465.000] - loss: 0.973 - mean_q: 9.616 - mean_eps: 1.000 - ale.lives: 2.171

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 11359s 1s/step - reward: 0.1890
done, took 79793.352 seconds


In [20]:
scores = dqn.test(env, nb_episodes = 10, visualize = True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 110.000, steps: 765
Episode 2: reward: 155.000, steps: 641
Episode 3: reward: 80.000, steps: 683
Episode 4: reward: 155.000, steps: 824
Episode 5: reward: 80.000, steps: 677
Episode 6: reward: 75.000, steps: 444
Episode 7: reward: 305.000, steps: 748
Episode 8: reward: 70.000, steps: 642
Episode 9: reward: 105.000, steps: 652
Episode 10: reward: 80.000, steps: 522
121.5


In [23]:
dqn.save_weights('models/dqn.h5f')

[WARNING] models/dqn.h5f.index already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


In [22]:
dqn.load_weights('models/dqn.h5f')